# YFinance Throughput & Soft Rate-Limit Analysis
This notebook measures:
- response latency  
- throughput  
- throttling signals  
without performing abusive load.


In [1]:
import time
import yfinance as yf
import numpy as np
from datetime import datetime
import pandas as pd

results = []


In [2]:
ticker = "MDLZ"
num_requests = 150          # enough to measure pattern
initial_sleep = 0.5         # 2 requests/sec
min_sleep = 0.05            # approach 20 req/sec gradually
sleep_decay = 0.97          # shrink sleep slightly each iteration


In [3]:
sleep_time = initial_sleep

for i in range(num_requests):

    t0 = time.time()
    try:
        df = yf.download(ticker, period="1d", interval="1m", progress=False)
        success = not df.empty
    except Exception as e:
        success = False

    latency = time.time() - t0

    results.append({
        "i": i,
        "success": success,
        "latency": latency,
        "sleep": sleep_time,
        "timestamp": datetime.now()
    })

    print(f"[{i}] success={success} latency={latency:.3f}s sleep={sleep_time:.3f}s")

    # detect throttling
    if not success:
        print("⚠️ Soft limit detected — stopping test safely.")
        break

    # gradually reduce sleep (increase RPS)
    sleep_time = max(min_sleep, sleep_time * sleep_decay)
    time.sleep(sleep_time)


YF.download() has changed argument auto_adjust default to True
[0] success=True latency=1.040s sleep=0.500s
[1] success=True latency=0.328s sleep=0.485s
[2] success=True latency=0.265s sleep=0.470s
[3] success=True latency=0.186s sleep=0.456s
[4] success=True latency=0.194s sleep=0.443s
[5] success=True latency=0.152s sleep=0.429s
[6] success=True latency=0.256s sleep=0.416s
[7] success=True latency=0.272s sleep=0.404s
[8] success=True latency=0.205s sleep=0.392s
[9] success=True latency=0.197s sleep=0.380s
[10] success=True latency=0.257s sleep=0.369s
[11] success=True latency=0.237s sleep=0.358s
[12] success=True latency=0.370s sleep=0.347s
[13] success=True latency=0.178s sleep=0.337s
[14] success=True latency=0.264s sleep=0.326s
[15] success=True latency=0.257s sleep=0.317s
[16] success=True latency=0.160s sleep=0.307s
[17] success=True latency=0.230s sleep=0.298s
[18] success=True latency=0.277s sleep=0.289s
[19] success=True latency=0.171s sleep=0.280s
[20] success=True latency=0

In [4]:
df = pd.DataFrame(results)
df.tail()


,i,success,latency,sleep,timestamp
145,145,True,0.146508,0.05,2025-12-02 00:07:22.926401
146,146,True,0.245162,0.05,2025-12-02 00:07:23.242427
147,147,True,0.276607,0.05,2025-12-02 00:07:23.569501
148,148,True,0.209660,0.05,2025-12-02 00:07:23.830105
149,149,True,0.190605,0.05,2025-12-02 00:07:24.071339


In [5]:
print("Average latency:", df['latency'].mean())
print("Min latency:", df['latency'].min())
print("Max latency:", df['latency'].max())

# where failures start
failures = df[~df['success']]
if len(failures) > 0:
    print("Soft throttle triggered at request:", failures.iloc[0]['i'])
    print("Sleep time at trigger:", failures.iloc[0]['sleep'])
else:
    print("No throttle encountered.")


Average latency: 0.250387765566508
Min latency: 0.13726353645324707
Max latency: 1.0399448871612549
No throttle encountered.
